In [ ]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import yaml

sys.path.append("../")
from globals import ACCURACY_METRICS, UNCERTAINTY_METRICS
from probabilistic.HMC.adv_robust_dp_hmc import AdvHamiltonianMonteCarlo
from probabilistic.HMC.hyperparams import HyperparamsHMC
from probabilistic.HMC.vanilla_bnn import VanillaBnnMnist
from common.dataset_utils import load_mnist
from globals import TORCH_DEVICE
from experiment_utils import test_ibp_acc_from_file

In [ ]:

#! 5 different models: dnn_sgd, hmc, fgsm_hmc, pgd_hmc, ibp_hmc
#! 8 metrics: std_acc, fgsm_acc, pgd_acc, ibp_acc, in_distrib_auroc, in_distrib_ece, ood_auroc, ood_ece
#? maybe privacy?
def plot_metrics_for_adv_comparison(dataset_name: str = "MNIST", metric_type = "accuracy"):
    # now fill them up with the results from the yaml file
    print(type(globals()['_dh'][0]))
    adv_results_file = str(globals()['_dh'][0]) + "/results_adv.yaml"
    experiments_results = None
    with open(adv_results_file, 'r', encoding="utf-8") as file:
        experiments_results = yaml.safe_load(file)[dataset_name]

    models = list(experiments_results.keys())
    positions = np.arange(0, len(models), 1)
    colors = ["#d46d3d", "#e3c268", "#ccf56e", "#64d199", "#68a0e3"]
    hlines = np.arange(0, 1.1, 0.1)

    desired_metrics = ACCURACY_METRICS if metric_type == "accuracy" else UNCERTAINTY_METRICS

    fig, ax = plt.subplots(2, 2, figsize=(30, 20), constrained_layout=True)
    for idx, metric in enumerate(desired_metrics):
        row, col = idx // 2, idx % 2
        heights = [experiments_results[model][metric] for model in models]
        ax[row, col].bar(positions, height=heights, width=0.5, align='center', color=colors, tick_label=models, alpha=0.9)
        ax[row, col].set_ylim(0, 1)
        ax[row, col].set_yticks(hlines)
        ax[row, col].hlines(hlines, -0.5, len(models) - 1 + 0.5, colors="gray", alpha=0.3)
        ax[row, col].set_title(metric)
        ax[row, col].set_ylabel("Accuracy")
        ax[row, col].set_xlabel("Training method")

    fig.show()
    plt.show()

In [ ]:
plot_metrics_for_adv_comparison("MNIST")

In [ ]:
plot_metrics_for_adv_comparison("MNIST", "uncertainty")

In [ ]:
vanilla_bnn = VanillaBnnMnist().to(TORCH_DEVICE)
train_data, test_data = load_mnist()
hyperparams = HyperparamsHMC(num_epochs=25, num_burnin_epochs=0, step_size=1e-2, warmup_step_size=0.2, lf_steps=120, batch_size=500, momentum_std=0.001,
                             alpha=0.5, eps=0.1, decay_epoch_start=20, lr_decay_magnitude=0.1, eps_warmup_epochs=0, alpha_warmup_epochs=0, run_dp=False)
hmc = AdvHamiltonianMonteCarlo(vanilla_bnn, hyperparams)
epsilons = [x / 10 for x in range(0, 1, 0.1)]
accs = test_ibp_acc_from_file(hmc, test_data, "posterior_samples/hmc_ibp.npy", epsilons)